## Observations and Insights 

In [129]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_data_complete = pd.merge(study_results, mouse_metadata, how='left', on=['Mouse ID', 'Mouse ID'])

# Display the data table for preview
mouse_data_complete.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [130]:
# Checking the number of mice
number_of_mice = mouse_metadata['Mouse ID'].count()
number_of_mice

249

In [131]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint
duplicated_mice_by_ID = study_results[study_results.duplicated(['Mouse ID', 'Timepoint'])]
duplicated_mice_by_ID.value_counts()

Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites
g989      0          45.000000           0                   1
          5          47.570392           0                   1
          10         49.880528           0                   1
          15         53.442020           0                   1
          20         54.657650           1                   1
dtype: int64

In [132]:
# Getting the duplicate mice by ID number and converting it into Series
duplicated_mice_ID = duplicated_mice_by_ID['Mouse ID'].unique()
duplicated_mice_ID = pd.Series(duplicated_mice_ID.squeeze())
duplicated_mice_ID

0    g989
dtype: object

In [133]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicated_mice_complete_data = mouse_data_complete[mouse_data_complete['Mouse ID'] == 'g989']
duplicated_mice_complete_data

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
107,g989,0,45.000000,0,Propriva,Female,21,26
137,g989,0,45.000000,0,Propriva,Female,21,26
329,g989,5,48.786801,0,Propriva,Female,21,26
360,g989,5,47.570392,0,Propriva,Female,21,26
620,g989,10,51.745156,0,Propriva,Female,21,26
681,g989,10,49.880528,0,Propriva,Female,21,26
815,g989,15,51.325852,1,Propriva,Female,21,26
869,g989,15,53.442020,0,Propriva,Female,21,26
950,g989,20,55.326122,1,Propriva,Female,21,26
1111,g989,20,54.657650,1,Propriva,Female,21,26


In [134]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID
mouse_data_complete_clean_df = mouse_data_complete[~ mouse_data_complete['Mouse ID'].isin(duplicated_mice_ID)]
mouse_data_complete_clean_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [135]:
# Checking the number of mice in the clean DataFrame
number_of_mice = mouse_data_complete_clean_df['Mouse ID'].nunique()
number_of_mice

249

## Summary Statistics

In [136]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume
# Assemble the resulting series into a single summary dataframe

drug_regimen = mouse_data_complete_clean_df.groupby('Drug Regimen')
mean = drug_regimen['Tumor Volume (mm3)'].mean()
median = drug_regimen['Tumor Volume (mm3)'].median()
variance = drug_regimen['Tumor Volume (mm3)'].var()
SD = drug_regimen['Tumor Volume (mm3)'].std()
SEM = drug_regimen['Tumor Volume (mm3)'].sem()

drug_regimen_summary = pd.DataFrame({'Mean': mean,
                                'Median': median,
                                'Variance': variance,
                                'Standard Deviation': SD,
                                'SEM': SEM
                                })
drug_regimen_summary

,Mean,Median,Variance,Standard Deviation,SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.322552,50.854632,42.351070,6.507770,0.512884
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [137]:
# Using the aggregation method, produce the same summary statistics in a single line

drug_regimen_summary_agg = mouse_data_complete_clean_df.groupby('Drug Regimen').agg({'Tumor Volume (mm3)': ['mean', 'median', 'var', 'std', 'sem']})
drug_regimen_summary_agg

Tumor Volume (mm3)                                          
                           mean     median        var       std       sem
Drug Regimen                                                             
Capomulin             40.675741  41.557809  24.947764  4.994774  0.329346
Ceftamin              52.591172  51.776157  39.290177  6.268188  0.469821
Infubinol             52.884795  51.820584  43.128684  6.567243  0.492236
Ketapril              55.235638  53.698743  68.553577  8.279709  0.603860
Naftisol              54.331565  52.509285  66.173479  8.134708  0.596466
Placebo               54.033581  52.288934  61.168083  7.821003  0.581331
Propriva              52.322552  50.854632  42.351070  6.507770  0.512884
Ramicane              40.216745  40.673236  23.486704  4.846308  0.320955
Stelasyn              54.233149  52.431737  59.450562  7.710419  0.573111
Zoniferol             53.236507  51.818479  48.533355  6.966589  0.516398

## Bar and Pie Charts

In [138]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.



In [139]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.



In [140]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas



In [141]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [142]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [143]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [144]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [145]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [146]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [147]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
